# Clasificador gaussiano ingenuo (Gaussian Naive Classifier)

In [3]:
import numpy as np

class GaussianNaiveBayes:
    def __init__(self, epsilon=1e-5):
        self.epsilon = epsilon
        self.probs = {}
        self.means = {}
        self.stds = {}

    def fit(self, X, y):
        classes = np.unique(y)
        for clase in classes:
            X_c = X[y == clase]
            self.probs[clase] = len(X_c) / len(X)
            self.means[clase] = np.mean(X_c, axis=0)
            self.stds[clase] = np.std(X_c, axis=0) + self.epsilon

    def calculate_likelihood(self, x, mean, std):
        p_x_given_y = (1 / ((np.sqrt(2 * np.pi) * std) + self.epsilon)) * np.exp(-((x - mean) ** 2) / ((2 * (std ** 2)) + self.epsilon))
        return p_x_given_y

    def predict(self, X):
        predictions = []
        for x in X:
            class_probs = {}
            for clase in self.probs:
                mean = self.means[clase]
                std = self.stds[clase]
                class_probs[clase] = np.log(self.probs[clase]) + np.sum(np.log(self.calculate_likelihood(x, mean, std)))

            predicted_class = max(class_probs, key=class_probs.get)
            predictions.append(predicted_class)

        return predictions

# Utilizando descriptor SIFT para clasificar

In [4]:
import cv2
import matplotlib.pyplot as plt

# Descriptor SIFT para extraer características
def puntos_clave(imagen, imagen_g):
    sift = cv2.SIFT_create()
    puntos_clave, descriptores = sift.detectAndCompute(imagen_g, None)
    #imagen_puntos_clave = cv2.drawKeypoints(imagen, puntos_clave, None)

    #plt.imshow(imagen_puntos_clave)
    #plt.title('Identificación de puntos clave')
    #plt.show()

    return puntos_clave, descriptores

In [5]:
comp_ruta = '../../IMAGES/'

In [6]:
unique_labels = ['flowers', 'glass_tiles', 'hairs', 'jelly_beans', 'oil_painting', 'paints', 'photo_and_face', 'pompoms', 'sponges', 'stuffed_toys']
descriptors_list = []
labels = []

for i in range(31):
    for j in range(10):
        # Abrir imagen
        if i < 9:
            ruta = comp_ruta+'complete_ms_data/'+str(unique_labels[j])+'/'+str(unique_labels[j])+'_ms_0'+str(i+1)+'.png'
        else:
            ruta = comp_ruta+'complete_ms_data/'+str(unique_labels[j])+'/'+str(unique_labels[j])+'_ms_'+str(i+1)+'.png'

        print(ruta)
        image = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
        # Convertir a RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Convertir a escala de grises
        image_g = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Extraer caracteristicas con SIFT
        keypoints, descriptor = puntos_clave(image, image_g)
        # Agregar descriptor al dataset
        descriptors_list.append(descriptor[0])
        labels.append(unique_labels[j])

../../IMAGES/complete_ms_data/flowers/flowers_ms_01.png
../../IMAGES/complete_ms_data/glass_tiles/glass_tiles_ms_01.png
../../IMAGES/complete_ms_data/hairs/hairs_ms_01.png
../../IMAGES/complete_ms_data/jelly_beans/jelly_beans_ms_01.png
../../IMAGES/complete_ms_data/oil_painting/oil_painting_ms_01.png
../../IMAGES/complete_ms_data/paints/paints_ms_01.png
../../IMAGES/complete_ms_data/photo_and_face/photo_and_face_ms_01.png
../../IMAGES/complete_ms_data/pompoms/pompoms_ms_01.png
../../IMAGES/complete_ms_data/sponges/sponges_ms_01.png
../../IMAGES/complete_ms_data/stuffed_toys/stuffed_toys_ms_01.png
../../IMAGES/complete_ms_data/flowers/flowers_ms_02.png
../../IMAGES/complete_ms_data/glass_tiles/glass_tiles_ms_02.png
../../IMAGES/complete_ms_data/hairs/hairs_ms_02.png
../../IMAGES/complete_ms_data/jelly_beans/jelly_beans_ms_02.png
../../IMAGES/complete_ms_data/oil_painting/oil_painting_ms_02.png
../../IMAGES/complete_ms_data/paints/paints_ms_02.png
../../IMAGES/complete_ms_data/photo_and_

In [8]:
unique_labels = ['flowers', 'glass_tiles', 'hairs', 'jelly_beans', 'oil_painting', 'paints', 'photo_and_face', 'pompoms', 'sponges', 'stuffed_toys']

for j in range(10):
    # Abrir imagen
    ruta = comp_ruta+'complete_ms_data/'+str(unique_labels[j])+'/'+str(unique_labels[j])+'_RGB'+'.bmp'

    print(ruta)
    image = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
    # Convertir a RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convertir a escala de grises
    image_g = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extraer caracteristicas con SIFT
    keypoints, descriptor = puntos_clave(image, image_g)
    # Agregar descriptor al dataset
    descriptors_list.append(descriptor[0])
    labels.append(unique_labels[j])

../../IMAGES/complete_ms_data/flowers/flowers_RGB.bmp


../../IMAGES/complete_ms_data/glass_tiles/glass_tiles_RGB.bmp
../../IMAGES/complete_ms_data/hairs/hairs_RGB.bmp
../../IMAGES/complete_ms_data/jelly_beans/jelly_beans_RGB.bmp
../../IMAGES/complete_ms_data/oil_painting/oil_painting_RGB.bmp
../../IMAGES/complete_ms_data/paints/paints_RGB.bmp
../../IMAGES/complete_ms_data/photo_and_face/photo_and_face_RGB.bmp
../../IMAGES/complete_ms_data/pompoms/pompoms_RGB.bmp
../../IMAGES/complete_ms_data/sponges/sponges_RGB.bmp
../../IMAGES/complete_ms_data/stuffed_toys/stuffed_toys_RGB.bmp


In [10]:
# Separar el dataset en 75% para entrenamiento y 25% para pruebas
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns

descriptors = np.array(descriptors_list)
labels = np.array(labels)

#X_train, X_test, y_train, y_test = descriptors[:240], descriptors[240:], labels[:240], labels[240:]
X_train, X_test, y_train, y_test = train_test_split(descriptors, labels, test_size=0.25, stratify=labels, random_state=42)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Mostrar los datos del descriptor y su clase en un dataframe
data= pd.DataFrame(descriptors)
data['Class'] = labels
data.head(10)

In [ ]:
# Mostrar el histograma de clases
# Se muestra que el numero de instancias es igual para todas las clases
data["Class"].hist()

In [ ]:
# Clasificamos con Gaussian Naive Bayes
gnb = GaussianNaiveBayes()
# Fit de data
gnb.fit(X_train, y_train)
# Predict
predicted = gnb.predict(X_test)

matriz = []
cont = 0
for tam in range(len(y_test)):
  matriz.append(y_test[tam] == predicted[tam])
  if y_test[tam] == predicted[tam]:
    cont = cont + 1

print(matriz)
percent = cont/len(y_test)*100
print("Accuracy {:.2f}%".format(percent))
print()

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print('Matriz de confusion:')
print(confusion_matrix(target_test, predicted))
percent = f1_score(target_test, predicted, average='weighted') * 100
print("Accuracy: {:.2f}%".format(percent))

# Utilizando descriptor HOG para clasificar

In [ ]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

unique_labels = ['flowers', 'glass_tiles', 'hairs', 'jelly_beans', 'oil_painting', 'paints', 'photo_and_face', 'pompoms', 'sponges', 'stuffed_toys']
descriptors_list = []
labels = []
for i in range(31):
    for j in range(10):
        # Abrir imagen
        if i < 9:
            ruta = comp_ruta+'complete_ms_data/'+str(unique_labels[j])+'/'+str(unique_labels[j])+'_ms_0'+str(i+1)+'.png'
        else:
            ruta = comp_ruta+'complete_ms_data/'+str(unique_labels[j])+'/'+str(unique_labels[j])+'_ms_'+str(i+1)+'.png'

        print(ruta)
        image = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
        # Convertir a RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Convertir a escala de grises
        image_g = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Extraer caracteristicas con HOG
        hog_features, _ = hog(image, orientations=8, pixels_per_cell=(16, 16),
                              cells_per_block=(1, 1), visualize=True, channel_axis=-1)
        # Agregar descriptor al dataset
        descriptors_list.append(hog_features)
        labels.append(unique_labels[j])

In [ ]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

unique_labels = ['flowers', 'glass_tiles', 'hairs', 'jelly_beans', 'oil_painting', 'paints', 'photo_and_face', 'pompoms', 'sponges', 'stuffed_toys']

for j in range(10):
    # Abrir imagen
    ruta = comp_ruta+'complete_ms_data/'+str(unique_labels[j])+'/'+str(unique_labels[j])+'_RGB'+'.bmp'

    print(ruta)
    image = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
    # Convertir a RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convertir a escala de grises
    image_g = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extraer caracteristicas con HOG
    hog_features, _ = hog(image, orientations=8, pixels_per_cell=(16, 16),
                              cells_per_block=(1, 1), visualize=True, channel_axis=-1)
    # Agregar descriptor al dataset
    descriptors_list.append(hog_features)
    labels.append(unique_labels[j])

In [ ]:
# Separar el dataset en 75% para entrenamiento y 25% para pruebas
descriptors = np.array(descriptors_list)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(descriptors, labels, test_size=0.25, stratify=labels, random_state=42)

In [ ]:
# Mostrar el dataframe con todos los datos
data = pd.DataFrame(descriptors)
data['Class'] = labels
data.head(10)

In [ ]:
# Clasificar con Gaussian Naive Bayes
gnb = GaussianNaiveBayes()
# Fit del data
gnb.fit(X_train, y_train)
# Predict
predicted = gnb.predict(X_test)

matriz = []
cont = 0
for tam in range(len(y_test)):
  matriz.append(y_test[tam] == predicted[tam])
  if y_test[tam] == predicted[tam]:
    cont = cont + 1

print(matriz)
percent = cont/len(y_test)*100
print("Accuracy {:.2f}%".format(percent))
print()

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
# Mostrar matriz de confusion y accuracy
print('Matriz de confusion:')
print(confusion_matrix(y_test, predicted))
percent = f1_score(y_test, predicted, average='weighted') * 100
print("Accuracy: {:.2f}%".format(percent))